In [1]:
import numpy as np
import pandas as pd


C:\Users\shivang\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\shivang\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\shivang\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_csv('../datasets/bayes/mushrooms.csv') 

In [3]:
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [4]:
df.shape

(8124, 23)

In [5]:
# converting numeric data
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [6]:
le = LabelEncoder()

In [7]:
ds = df.apply(func = le.fit_transform) # this func will be apllied to each adn every column

In [8]:
ds.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [9]:
data = ds.values # gives the numpy arrray

In [10]:
X = data[:,1:]
y = data[:,0]

In [11]:
X.shape, y.shape

((8124, 22), (8124,))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
X_train

array([[2, 0, 2, ..., 2, 5, 0],
       [5, 0, 3, ..., 2, 4, 0],
       [5, 0, 2, ..., 3, 5, 0],
       ...,
       [2, 3, 4, ..., 3, 5, 4],
       [3, 2, 2, ..., 7, 4, 4],
       [3, 0, 3, ..., 7, 2, 1]])

In [14]:
# Prior theory
np.array([2,2,3,2,3]) == np.array([2,3,3,2,2])

array([ True, False,  True,  True, False])

In [15]:
np.sum(np.array([2,2,3,2,3]) == np.array([2,3,3,2,2])) # total number of trues

3

In [16]:
X_test

array([[2, 0, 4, ..., 3, 3, 1],
       [2, 2, 2, ..., 7, 4, 2],
       [5, 3, 4, ..., 7, 4, 2],
       ...,
       [5, 0, 4, ..., 2, 5, 0],
       [5, 3, 7, ..., 1, 4, 0],
       [2, 0, 4, ..., 3, 5, 0]])

In [17]:
X_test[0]

array([2, 0, 4, 0, 5, 1, 1, 0, 3, 1, 3, 2, 0, 7, 7, 0, 2, 1, 0, 3, 3, 1])

In [18]:
# Custom NB
class CustomNB:
    def fit(self,X,y):
        self.X_train = X
        self.y_train = y
    
    # label = which class we want this prob for
    
    def prior_prob(self,label):
        total = self.y_train.shape[0]
        class_examples =np.sum(self.y_train==label)
        return class_examples/float(total) 
    
   # P(Xi =red | y =label) -ith featres (feature col = i) for a single example
    
    def conditional_prob(self,feature_col,feature_val, label):
        # out of all examples what mushrooom have features as feature-vval in the feature-col that belongs to that class
        X_filtered = self.X_train[self.y_train==label]
        numerator = np.sum(X_filtered[:,feature_col]==feature_val)
        denominator = len(X_filtered)
        return numerator/denominator
    
    # we are going to do this for all the 22 faetures we have
    def predict_point(self,X_test):
        # X_test is a class witht n-features
        classes = np.unique(self.y_train)
        n_features = self.X_train.shape[1]
        post_pro = []
        # posterior probability for each class
        
        for label in classes:
            # post_probb = prior * likelihood
            likehood = 1.0
            for feature in range(n_features):
                cond = self.conditional_prob(feature, X_test[feature],label)
                likehood *= cond
            
            prior = self.prior_prob(label)
            post = prior * likehood
            post_pro.append(post)
        
        # ans = max value from all the labels
        return np.argmax(post_pro) # return the index of largest value in arr
    
    def predict(self, X_test):
        res = []
        for point in X_test:
            res.append(self.predict_point(point))
        return np.array(res)
    
    def score(self,X_test,y_test):
        return (self.predict(X_test)==y_test).mean()     

In [19]:
model = CustomNB()

In [20]:
model.fit(X_train,y_train)

In [21]:
model.predict(X_test[:10])

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0], dtype=int64)

In [22]:
y_test[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [23]:
model.score(X_test,y_test)

0.9973890339425587